In [34]:
import subprocess as sp
import gurobipy as gp
import sage.all as sg
from sage.all import libgap as lg
import re

In [35]:
def write_orbits(orbits):
    with open("./sage_orbits.txt", "w") as f:
        idx = 0
        for orbit in orbits:
            for el in orbit:
                f.write(str(el - 1) + " ")
            f.write("\n")
            idx += 1
            
            
def get_stabilizer_group_orbits(group):
    group_orbits = group.orbits()
    for orb in group_orbits:
        if len(orb) > 1:
            stable_pnt = orb[0]
            stabilizer_group = group.stabilizer(stable_pnt)
            return stabilizer_group, stabilizer_group.orbits()
    return group, group.orbits()
        
def concatenate_orbits(real_orbits, cut_orbits):
    out = []
    for orb in real_orbits:
        out.append(orb)
    for orb in cut_orbits:
        out.append(orb)
    return out

def call_coin_cgl(lp_path):
    cgl_cut_file = "./CutGeneration/cut_txt_files/cuts.txt"
    cgl_out = sp.Popen(["./CutGeneration/generateMIPCuts",lp_path,"./sage_orbits.txt"], shell = False, 
                       stdout = sp.DEVNULL)
    cgl_return, cgl_err = cgl_out.communicate()
    return cgl_return, cgl_err

def generate_cut_orbits(gap_perm_group, lp_path, cut_orbits):
    cut_rhs = []
    cut_added_model = gp.read(lp_path)
    cols = cut_added_model.getVars()
    rows = cut_added_model.getConstrs()
    num_nodes = len(cols) + len(rows) + 1
    with open("./CutGeneration/cut_txt_files/cuts.txt") as cuts_f:
        lines = cuts_f.readlines()
        for line in lines:
            splt = line.split(",")
            cut = []
            rhs = (sg.RealNumber(splt[-1][:-1]))
            sense = splt[-2]
            for coeff in splt[:-2]:
                cut.append(sg.RealNumber(coeff))
            symmetric_cuts = lg.Orbit(gap_perm_group, cut, lg.Permuted)
            new_orbits = []
            for sym_cut in symmetric_cuts:
                if sense == "L":
                    expr = sum(sym_cut[i].sage() * orig_cols[i] for i in range(len(sym_cut))) <= rhs
                elif sense == "G":
                    expr = sum(sym_cut[i].sage() * orig_cols[i] for i in range(len(sym_cut))) >= rhs
                else:
                    expr = sum(sym_cut[i].sage() * orig_cols[i] for i in range(len(sym_cut))) == rhs
                orig_model.addConstr(expr)
                new_orbits.append(num_nodes)
                num_nodes += 1
            cut_orbits.append(new_orbits)
    orig_model.write("test.mps")
    orig_model.write("test.lp")

In [36]:
# Read in mps file and create dicitonary for variable names and indices 
orig_model = gp.read("../EQLPSolver/HiGHS-1-2-1/smallTests/codbt021.mps")
orig_cols = orig_model.getVars()
orig_rows = orig_model.getConstrs()
col_idx = {col.varName : i for i, col in enumerate(orig_cols)}
col_name = {i : col.varName for i, col in enumerate(orig_cols)}
num_col = orig_model.numVars
row_idx = {row.constrName : i + num_col for i , row in enumerate(orig_rows)}
row_name = {i + num_col : row.constrName for i, row in enumerate(orig_rows)}
num_tot = len(orig_cols) + len(orig_rows)


Read MPS format model from file ../EQLPSolver/HiGHS-1-2-1/smallTests/codbt021.mps
Reading time = 0.00 seconds
: 9 rows, 9 columns, 45 nonzeros


In [37]:
# Call saucy on the lp graph
saucy_out_file = open("./saucy_gens.txt", "w")
saucy_out = sp.Popen(["./lp_saucy/bin/saucy2-5","../EQLPSolver/HiGHS-1-2-1/smallTests/codbt021.mps"], stdout = saucy_out_file,
                            stderr = saucy_out_file, shell = False)
saucy_return, saucy_err = saucy_out.communicate()
sage_gens = []
gen_idx = 0
# Read in the orbit generators from saucy
with open("./saucy_gens.txt") as f:
    lines = f.readlines()
    for line in lines:
        if line[0] == "C":
            continue
        else:
            sage_gens.append([])
            orbits = re.findall(r"\((.*?)\)",line)
            for orb in orbits:
                temp = []
                for node in orb.split(" "):
                    if node in col_idx.keys():
                        temp.append(col_idx.get(node) + 1)
                    else:
                        temp.append(row_idx.get(node) + 1)
                temp = tuple(temp)
                sage_gens[gen_idx].append(temp)
            gen_idx += 1

            
# Do sage permutation group from the generators
sage_perm_group = PermutationGroup(sage_gens)
gap_group = lg(sage_perm_group)
orbits_to_write = []
cut_orbits = []




In [38]:
# Get first stabilizer group and orbits
stab_group, real_orbits = get_stabilizer_group_orbits(sage_perm_group)
# Concatenate the orbits from the current stabilizer group
# and from the generated cuts and group action on said cuts
orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
# Write the orbits to a text file to be read into aggregation code
write_orbits(orbits_to_write)
# Call coin-or cut generator that generates cuts from my aggregate lp
call_coin_cgl("../EQLPSolver/HiGHS-1-2-1/smallTests/codbt021.mps")
# Generate symmetrical cuts in the orbit of cuts from coin-or cgl
generate_cut_orbits(gap_group, "../EQLPSolver/HiGHS-1-2-1/smallTests/codbt021.mps", cut_orbits)
# Run until the orbits are discrete
while (len(real_orbits) < num_tot):
    stab_group, real_orbits = get_stabilizer_group_orbits(stab_group)
    orbits_to_write = concatenate_orbits(real_orbits, cut_orbits)
    write_orbits(orbits_to_write)
    call_coin_cgl("./test.mps")
    generate_cut_orbits(gap_group, "./test.mps", cut_orbits)
all_cuts_added = gp.read("./test.mps")
all_cols = all_cuts_added.getVars()
for col in all_cols:
    col.VType = 'B'
all_cuts_added.optimize() 

Read MPS format model from file ../EQLPSolver/HiGHS-1-2-1/smallTests/codbt021.mps
Reading time = 0.00 seconds
: 9 rows, 9 columns, 45 nonzeros
Read MPS format model from file ./test.mps
Reading time = 0.00 seconds
: 19 rows, 9 columns, 135 nonzeros
Read MPS format model from file ./test.mps
Reading time = 0.00 seconds
: 70 rows, 9 columns, 594 nonzeros
Read MPS format model from file ./test.mps
Reading time = 0.00 seconds
: 88 rows, 9 columns, 756 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 88 rows, 9 columns and 756 nonzeros
Model fingerprint: 0x3a158015
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 3.0000000
Presolve removed 30 rows and 0 columns
Presolve time: 0.00s

In [113]:
old = gp.read("./test.mps")
# old.write("./test.lp")
old_x = old.getVars()
for x in old_x:
    x.VType = 'B'
old.optimize()

Read MPS format model from file ./test.mps
Reading time = 0.00 seconds
: 27 rows, 9 columns, 203 nonzeros
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 27 rows, 9 columns and 203 nonzeros
Model fingerprint: 0xc30cf5fe
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 3.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 26 rows, 9 columns, 193 nonzeros
Variable types: 0 continuous, 9 integer (9 binary)

Root relaxation: cutoff, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         